<a href="https://colab.research.google.com/github/APstats-labs/labs/blob/main/SA_C5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data pre-processing methods (3)

General requirements

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

Mounted at /content/drive


##Inter-rater agreement

### Preparing the data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
Evaluator1 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Statistics/Curs/Data_c2/SNLI_Anca.xlsx')
Evaluator2 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Statistics/Curs/Data_c2/SNLI_Ionut.xlsx')

In [4]:
Evaluator1_Missing = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Statistics/Curs/Data_c2/Missing_SNLI_Anca.xlsx')
Evaluator2_Missing = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Statistics/Curs/Data_c2/Missing_SNLI_Ionut.xlsx')

In [5]:
Aggregated_Missing = pd.concat([Evaluator1_Missing['Label'],Evaluator2_Missing['Label']], axis=1)

Aggregated_Missing = Aggregated_Missing.set_axis([ 'Evaluator1', 'Evaluator2'], axis=1)

In [6]:
Aggregated = pd.concat([Evaluator1['Label'],Evaluator2['Label']], axis=1)

Aggregated = Aggregated.set_axis([ 'Evaluator1', 'Evaluator2'], axis=1)

Aggregated

,Evaluator1,Evaluator2
0,Neutral,Contradiction
1,Neutral,Neutral
2,Entailment,Entailment
3,Contradiction,Contradiction
4,Neutral,Neutral
...,...,...
895,Entailment,Entailment
896,Neutral,Neutral
897,Neutral,Entailment
898,Contradiction,Contradiction


### Percent agreement

In [7]:
Agreement_table = pd.crosstab(Aggregated['Evaluator1'],
                          Aggregated['Evaluator2'],
                          margins = True)

display(Agreement_table.style.set_table_styles([{
    'selector': 'th',
    'props': [('background-color', 'lightgrey')]
}]))

Evaluator2,Contradiction,Entailment,Neutral,All
Evaluator1,,,,
Contradiction,166,18,18,202
Entailment,19,276,30,325
Neutral,136,62,175,373
All,321,356,223,900


Compute the percent agreement as the sum of the diagonal over the total.

In [8]:
(166+276+175)/900

0.6855555555555556

### Cohen's Kappa

In [9]:
#pip install -U scikit-learn
from sklearn.metrics import cohen_kappa_score

In [10]:
Rater_1 = Aggregated['Evaluator1']
Rater_2 = Aggregated['Evaluator2']
cohen_kappa_score(Rater_1, Rater_2)

0.5337547297260191

####Cohen's Kappa is done "by hand" 🤚

**Get the relative frequency table**

In [11]:
Agreement_table/900

Evaluator2,Contradiction,Entailment,Neutral,All
Evaluator1,,,,
Contradiction,0.184444,0.020000,0.020000,0.224444
Entailment,0.021111,0.306667,0.033333,0.361111
Neutral,0.151111,0.068889,0.194444,0.414444
All,0.356667,0.395556,0.247778,1.000000


**Compute the observed agreement**

In [12]:
P_agreement_observed = sum([0.18444, 0.306667, 0.194444])
P_agreement_observed

0.685551

**Compute the chance agreement**

In [13]:
P_Random_Contradiction = 0.356667 * 0.224444 #Probability that Rater 1 says Contradiction and Rater 2 also says Contradiction
P_Random_Contradiction

0.08005176814800001

In [14]:
P_Random_Entailment = 0.395556 * 0.361111 #Probability that Rater 1 says Entailment and Rater 2 also says Entailment
P_Random_Entailment

0.142839622716

In [15]:
P_Random_Neutral = 0.247778	 * 	0.414444 #Probability that Rater 1 says Neutral and Rater 2 also says Neutral
P_Random_Neutral

0.102690105432

In [16]:
P_random_expected = P_Random_Contradiction + P_Random_Entailment+P_Random_Neutral
P_random_expected

0.325581496296

**Include the observed and chance agreement computed previously into the formula**.

Expect minor differences in decimals due to rounding.

In [17]:
(P_agreement_observed -P_random_expected)/(1-P_random_expected)

0.5337479646940254

#### Cohen Kappa with missing Data ❓

Notice that pd crosstabs does not display Missing data.

In [18]:
Agreement_table_missing = pd.crosstab(Aggregated_Missing['Evaluator1'],
                          Aggregated_Missing['Evaluator2'],
                          margins = True)

display(Agreement_table_missing.style.set_table_styles([{
    'selector': 'th',
    'props': [('background-color', 'lightgrey')]
}]))

Evaluator2,Contradiction,Entailment,Neutral,All
Evaluator1,,,,
Contradiction,158,17,17,192
Entailment,18,269,26,313
Neutral,131,61,173,365
All,307,347,216,870


As such, we have to use fill.na with "missing" in order to display them

In [19]:
Agreement_table_missing = pd.crosstab(Aggregated_Missing['Evaluator1'].fillna('missing'),
                          Aggregated_Missing['Evaluator2'].fillna('missing'),
                          margins = True)

display(Agreement_table_missing.style.set_table_styles([{
    'selector': 'th',
    'props': [('background-color', 'lightgrey')]
}]))

Evaluator2,Contradiction,Entailment,Neutral,missing,All
Evaluator1,,,,,
Contradiction,158,17,17,5,197
Entailment,18,269,26,3,316
Neutral,131,61,173,7,372
missing,5,7,3,0,15
All,312,354,219,15,900


Computing Cohen's Kappa without adressing missing data leads to err

In [20]:
#from sklearn.metrics import cohen_kappa_score
#Rater_1_missing = Aggregated_Missing['Evaluator1']
#Rater_2_missing = Aggregated_Missing['Evaluator2']
#cohen_kappa_score(Rater_1_missing, Rater_2_missing)

Replacing Missing data with category "missing" leads to treating Missing values as categories. This just one of the existing approaches. Read more here: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6506991/

In [21]:
from sklearn.metrics import cohen_kappa_score
Rater_1_missing = Aggregated_Missing['Evaluator1'].fillna('missing')
Rater_2_missing = Aggregated_Missing['Evaluator2'].fillna('missing')
cohen_kappa_score(Rater_1_missing, Rater_2_missing)

0.5134951052201975

#### Weighted Cohen's Kappa 🏋

There are two options for weighting Kappa in sklearn: linear and quadratic. Read more here: https://stats.stackexchange.com/questions/59798/quadratic-weighted-kappa-versus-linear-weighted-kappa

Other weighting schemes can be accesed via other packages.For instance, https://pypi.org/project/agreement/

In [22]:
Agreement_table = pd.crosstab(Aggregated['Evaluator1'],
                          Aggregated['Evaluator2'],
                          margins = True)


In [23]:
from sklearn.metrics import cohen_kappa_score
Rater_1 = Aggregated['Evaluator1']
Rater_2 = Aggregated['Evaluator2']
cohen_kappa_score(Rater_1, Rater_2, weights='quadratic')

0.35567010309278346

In [24]:
Rater_1 = Aggregated['Evaluator1']
Rater_2 = Aggregated['Evaluator2']
cohen_kappa_score(Rater_1, Rater_2, weights='linear')

0.4468805638921157

##### Weighted Cohen's Kappa quadratic by hand 🤚

**Get observed frequencies**

In [25]:
observed_frequencies = Agreement_table.iloc[:-1, :-1]
observed_frequencies_display=observed_frequencies/900

display(observed_frequencies_display.style.set_table_styles([{
    'selector': 'th',
    'props': [('background-color', 'lightgrey')]
}]))

Evaluator2,Contradiction,Entailment,Neutral
Evaluator1,,,
Contradiction,0.184444,0.020000,0.020000
Entailment,0.021111,0.306667,0.033333
Neutral,0.151111,0.068889,0.194444


**Compute expected frequencies**

In [26]:
Agreement_relative = Agreement_table/900
Agreement_relative

Evaluator2,Contradiction,Entailment,Neutral,All
Evaluator1,,,,
Contradiction,0.184444,0.020000,0.020000,0.224444
Entailment,0.021111,0.306667,0.033333,0.361111
Neutral,0.151111,0.068889,0.194444,0.414444
All,0.356667,0.395556,0.247778,1.000000


In [27]:
# Remove the last row and column for calculations
observed = Agreement_relative.iloc[:-1, :-1]

# Extract row totals and column totals
row_totals = Agreement_relative['All'][:-1]
col_totals = Agreement_relative.loc['All'][:-1]
grand_total = Agreement_relative.loc['All', 'All']

# Initialize the expected frequencies DataFrame
expected_frequencies = pd.DataFrame(index=observed.index, columns=observed.columns)

# Calculate the expected frequencies
for row in observed.index:
    for col in observed.columns:
        expected_frequencies.loc[row, col] = (row_totals[row] * col_totals[col]) / grand_total

expected_frequencies

display(expected_frequencies.style.set_table_styles([{
    'selector': 'th',
    'props': [('background-color', 'lightgrey')]
}]))

Evaluator2,Contradiction,Entailment,Neutral
Evaluator1,,,
Contradiction,0.080052,0.088780,0.055612
Entailment,0.128796,0.142840,0.089475
Neutral,0.147819,0.163936,0.102690


**Create the weights matrix**

In [28]:
weights = np.array([
    [1, 0.75, 0],
    [0.75, 1, 0.75],
    [0, 0.75, 1]
])


**Compute weighted observed agreement**
Notice that now all the numbers in matrix are summed instead of diagonal. This is due to that of turning the Agreement/Disagreement matrix into an Agreement Matrix (where the values range from Full Agreement to No agreement, rather than to Dissagreement)

In [29]:
Chance_Agreement = expected_frequencies * weights
Chance_Agreement = Chance_Agreement.to_numpy().sum()
Chance_Agreement


0.6788222222222222

**Compute weighted chance agreement**

In [30]:
Observed_Agreement = observed_frequencies/900 * weights
Observed_Agreement = Observed_Agreement.to_numpy().sum()
Observed_Agreement

0.7930555555555555

**Compute kappa weighted (quadratic)**

In [31]:
(Observed_Agreement-Chance_Agreement)/(1-Chance_Agreement)

0.3556701030927834

### Scott PI

**First turn the data into numpy long format**

In [32]:
Aggregated['id'] = range(1, 901)

In [33]:
Aggregated_long_format = pd.melt(Aggregated, id_vars=['id'],
                      value_vars=['Evaluator1', 'Evaluator2'],
                      var_name='Rater', value_name='Answer')

# Display the resulting DataFrame
Aggregated_long_format = Aggregated_long_format.to_numpy()

Aggregated_long_format

array([[1, 'Evaluator1', 'Neutral'],
       [2, 'Evaluator1', 'Neutral'],
       [3, 'Evaluator1', 'Entailment'],
       ...,
       [898, 'Evaluator2', 'Entailment'],
       [899, 'Evaluator2', 'Contradiction'],
       [900, 'Evaluator2', 'Contradiction']], dtype=object)

**Then turn the data into an acceptable format for package "agreement"** (https://pypi.org/project/agreement/).
The output is a matrix in which rows are the evaluation objects (in this case, 900) and columns are the evaluation categories (in this case, Entailment, Contradiction, Neutral)

In [34]:
!pip install agreement

In [35]:
from agreement.utils.transform import pivot_table_frequency

questions_answers_table = pivot_table_frequency(Aggregated_long_format[:, 0], Aggregated_long_format[:, 2])
#users_answers_table = pivot_table_frequency(Aggregated_long_format[:, 1], Aggregated_long_format[:, 2]) this is used for cohen's Kappa but we already used the sklearn approach.
questions_answers_table

array([[1., 0., 1.],
       [0., 0., 2.],
       [0., 2., 0.],
       ...,
       [0., 1., 1.],
       [2., 0., 0.],
       [2., 0., 0.]])

**Then use the function "scotts_pi"**

In [36]:
from agreement.metrics import scotts_pi
scotts_pi(questions_answers_table)

0.5255865674129939

#### Scott Pi done "by hand" 🤚

**Compute observed agreement**

In [37]:
Agreement_table = pd.crosstab(Aggregated['Evaluator1'],
                          Aggregated['Evaluator2'],
                          margins = True)

display(Agreement_table.style.set_table_styles([{
    'selector': 'th',
    'props': [('background-color', 'lightgrey')]
}]))

Evaluator2,Contradiction,Entailment,Neutral,All
Evaluator1,,,,
Contradiction,166,18,18,202
Entailment,19,276,30,325
Neutral,136,62,175,373
All,321,356,223,900


In [38]:
P_scott_observed = (166+276+175)/900

**Compute chance Agreement**

First extract the margin totals and place them one next to each other.
A cell in the resulting table displays the frequency of choosing one category by a specific rater

In [39]:
# Extract the marginal totals
marginal_totals_evaluator2 = Agreement_table.loc['All', :]
marginal_totals_evaluator1 = Agreement_table['All']

# Form a new DataFrame with the extracted totals
Chance_agreement_table = pd.DataFrame({
    'Evaluator1': marginal_totals_evaluator1[:-1],
    'Evaluator2': marginal_totals_evaluator2[:-1]
}).reset_index(drop=True)

Chance_agreement_table .insert(0, 'Category', ["Contradiction", "Entailment", "Neutral"])

display(Chance_agreement_table.style.set_table_styles([{
    'selector': 'th',
    'props': [('background-color', 'lightgrey')]
}]))

,Category,Evaluator1,Evaluator2
0,Contradiction,202,321
1,Entailment,325,356
2,Neutral,373,223


Get the relative frequency of each category

In [40]:
Chance_agreement_table['Fqv'] = Chance_agreement_table[['Evaluator1', 'Evaluator2']].sum(axis=1)
Chance_agreement_table['Relative_Fqv'] = Chance_agreement_table[['Evaluator1', 'Evaluator2']].sum(axis=1) / 1800

display(Chance_agreement_table.style.set_table_styles([{
    'selector': 'th',
    'props': [('background-color', 'lightgrey')]
}]))

,Category,Evaluator1,Evaluator2,Fqv,Relative_Fqv
0,Contradiction,202,321,523,0.290556
1,Entailment,325,356,681,0.378333
2,Neutral,373,223,596,0.331111


Square the relative frequencies and add them

In [41]:
Chance_agreement_table['SquaredFqv'] = Chance_agreement_table['Relative_Fqv'] ** 2

display(Chance_agreement_table.style.set_table_styles([{
    'selector': 'th',
    'props': [('background-color', 'lightgrey')]
}]))

,Category,Evaluator1,Evaluator2,Fqv,Relative_Fqv,SquaredFqv
0,Contradiction,202,321,523,0.290556,0.084423
1,Entailment,325,356,681,0.378333,0.143136
2,Neutral,373,223,596,0.331111,0.109635


Get chance agreement

In [42]:
P_scott_chance = Chance_agreement_table['SquaredFqv'].sum()
P_scott_chance

0.3371932098765433

**Include the observed and chance agreement computed previously into the formula**.

In [43]:
(P_scott_observed - P_scott_chance)/(1 - P_scott_chance)

0.5255865674129939

### Gwet Gamma (AC1)

**First transform the data into an acceptable format (same as for Scott Pi)**

In [44]:
#!pip install agreement
Aggregated['id'] = range(1, 901)

Aggregated_long_format = pd.melt(Aggregated, id_vars=['id'],
                      value_vars=['Evaluator1', 'Evaluator2'],
                      var_name='Rater', value_name='Answer')

Aggregated_long_format = Aggregated_long_format.to_numpy()

from agreement.utils.transform import pivot_table_frequency

questions_answers_table = pivot_table_frequency(Aggregated_long_format[:, 0], Aggregated_long_format[:, 2])

**Then use the "gwets_gamma" function**

In [45]:
from agreement.metrics import gwets_gamma
gwets_gamma(questions_answers_table)

0.5296948232731818

#### Gwet's gamma done by hand 🤚

**Compute oberved agreement**

In [46]:
P_gamma_o = (166+276+175)/900

**Compute chance agreement**

Notice it uses the same table as for Scott Pi except there is not "Squared Fqv colum"

In [47]:
# Extract the marginal totals
marginal_totals_evaluator2 = Agreement_table.loc['All', :]
marginal_totals_evaluator1 = Agreement_table['All']

# Form a new DataFrame with the extracted totals
Chance_agreement_table = pd.DataFrame({
    'Evaluator1': marginal_totals_evaluator1[:-1],
    'Evaluator2': marginal_totals_evaluator2[:-1]
}).reset_index(drop=True)

Chance_agreement_table .insert(0, 'Category', ["Contradiction", "Entailment", "Neutral"])

Chance_agreement_table['Fqv'] = Chance_agreement_table[['Evaluator1', 'Evaluator2']].sum(axis=1)
Chance_agreement_table['Relative_Fqv'] = Chance_agreement_table[['Evaluator1', 'Evaluator2']].sum(axis=1) / 1800

display(Chance_agreement_table.style.set_table_styles([{
    'selector': 'th',
    'props': [('background-color', 'lightgrey')]
}]))

,Category,Evaluator1,Evaluator2,Fqv,Relative_Fqv
0,Contradiction,202,321,523,0.290556
1,Entailment,325,356,681,0.378333
2,Neutral,373,223,596,0.331111


In [48]:
P_gamma_c = ((0.290556 * (1-0.290556)) + (0.378333	*(1-0.378333) + 0.331111 * (1-0.331111)))/(3-1)
P_gamma_c

0.33140342882699997

**Insert the observed and chance agreement computed previously into the formula**.

In [49]:
(P_gamma_o - P_gamma_c )/(1 - P_gamma_c )

0.5296947995219653

### Kipendorff Alpha

**First transform the data into an acceptable format (same as for Scott Pi & Gwet Gamma)**

In [50]:
#!pip install agreement
agr = Aggregated.copy()
agr['id'] = range(1, 901)

Aggregated_long_format = pd.melt(agr, id_vars=['id'],
                      value_vars=['Evaluator1', 'Evaluator2'],
                      var_name='Rater', value_name='Answer')

Aggregated_long_format = Aggregated_long_format.to_numpy()


from agreement.utils.transform import pivot_table_frequency

questions_answers_table = pivot_table_frequency(Aggregated_long_format[:, 0], Aggregated_long_format[:, 2])

**Then use "krippendorffs_alpha" function**

In [51]:
from agreement.metrics import krippendorffs_alpha
krippendorffs_alpha(questions_answers_table)

0.525850130431098

#### Kipendorff Alpha by hand 🤚

**Approach using coincidence matrix**

First get the coincidence matrix

❗Notice that this is different from the contingency table that was computed in percent agreement. It shows the number of pairable values, not the number of items labeled as such by the raters.

In [53]:
# Convert to DataFrame
df = pd.DataFrame(Aggregated)
df = df.replace('*', np.nan)

# Initialize the coincidence matrix with float dtype
labels = ["Contradiction", "Entailment", "Neutral"]
coincidence_matrix = pd.DataFrame(0.0, index=labels, columns=labels, dtype=float)

# Iterate over each pair (i, j)
for i in labels:
    for j in labels:
        if i != j:
            # Find rows where both i and j are present in the same row
            rows_with_ij = df[(df == i).any(axis=1) & (df == j).any(axis=1)]

            # Initialize a Series to store i_j_pairs_count for each row
            i_j_pairs_count_per_row = pd.Series(0.0, index=rows_with_ij.index)

            # Iterate over each row in rows_with_ij
            for index, row in rows_with_ij.iterrows():
                # Count occurrences of i and j in the current row
                count_i = row.eq(i).sum()
                count_j = row.eq(j).sum()

                # Calculate the number of i-j pairs for the current row
                i_j_pairs_count = count_i * count_j

                non_nan_counts = rows_with_ij.notna().sum(axis=1)
                # Store the result in the Series
                i_j_pairs_count_per_row[index] = float(i_j_pairs_count) / (non_nan_counts[index] - 1)

            # Store the i_j_pairs_count_per_row in the coincidence matrix
            coincidence_matrix.at[i, j] = float(i_j_pairs_count_per_row.sum())

        else:
            # Find rows where both i and j appear more than once in the same row
            rows_with_iAj = df[(df == i).sum(axis=1) > 1]

            # Initialize a Series to store i_A_j_pairs_count for each row
            i_A_j_pairs_count_per_row = pd.Series(0.0, index=rows_with_iAj.index)

            # Iterate over each row in rows_with_ij
            for index, row in rows_with_iAj.iterrows():
                # Count occurrences of i and j in the current row
                count_i = row.eq(i).sum()

                # Calculate the number of i-j pairs for the current row
                i_A_j_pairs_count = count_i * (count_i - 1)

                non_nan_counts_I = rows_with_iAj.notna().sum(axis=1)
                # Store the result in the Series
                i_A_j_pairs_count_per_row[index] = float(i_A_j_pairs_count) / (non_nan_counts_I[index] - 1)

            # Store the i_A_j_pairs_count_per_row in the coincidence matrix
            coincidence_matrix.at[i, j] = float(i_A_j_pairs_count_per_row.sum())

# Adding totals
coincidence_matrix["Total"] = coincidence_matrix.sum(axis=1)
coincidence_matrix.loc["Total"] = coincidence_matrix.sum(axis=0)

# Display the styled coincidence matrix
display(coincidence_matrix.style.set_table_styles([{
    'selector': 'th',
    'props': [('background-color', 'lightgrey')]
}]))


,Contradiction,Entailment,Neutral,Total
Contradiction,166.000000,18.500000,77.000000,261.500000
Entailment,18.500000,276.000000,46.000000,340.500000
Neutral,77.000000,46.000000,175.000000,298.000000
Total,261.500000,340.500000,298.000000,900.000000


Then compute the observed agreement, chance agreement, and insert into the formula

❗Notice that the numbers below are obtained by doubling the row values in the coincidence matrix

In [ ]:
n = 2*len(Aggregated)
ObservedAgreement = (332+552+350)/n
ChanceAgreement = (
    ((332 +  37 + 154) * (332 +  37 + 154 - 1) +
     (37 + 552 + 92) * (37 + 552 + 92 - 1) +
     (154 + 92 + 350) * (154 + 92 + 350-1)) /
    (n * (n - 1))
)
(ObservedAgreement-ChanceAgreement)/(1-ChanceAgreement)

0.525850130431098

In [ ]:
ChanceAgreement

0.33682477919832005

**Approach using relative frequencies**

After computing the precent agreement, generate the table below

In [ ]:
Agreement_table = pd.crosstab(Aggregated['Evaluator1'],
                          Aggregated['Evaluator2'],
                          margins = True)

# Extract the marginal totals
marginal_totals_evaluator2 = Agreement_table.loc['All', :]
marginal_totals_evaluator1 = Agreement_table['All']

# Form a new DataFrame with the extracted totals
Chance_agreement_table_3 = pd.DataFrame({
    'Evaluator1': marginal_totals_evaluator1[:-1],
    'Evaluator2': marginal_totals_evaluator2[:-1]
}).reset_index(drop=True)

Chance_agreement_table_3.insert(0, 'Category', ["Contradiction", "Entailment", "Neutral"])

Chance_agreement_table_3['Fqv'] = Chance_agreement_table_3[['Evaluator1', 'Evaluator2']].sum(axis=1)

Chance_agreement_table_3['Fqv_Pairs'] = Chance_agreement_table_3['Fqv'] * (Chance_agreement_table_3['Fqv']-1)

Chance_agreement_table_3['Relative_Fqv_Pairs'] = Chance_agreement_table_3['Fqv_Pairs']/(1800*(1800-1))

display(Chance_agreement_table_3.style.set_table_styles([{
    'selector': 'th',
    'props': [('background-color', 'lightgrey')]
}]))

,Category,Evaluator1,Evaluator2,Fqv,Fqv_Pairs,Relative_Fqv_Pairs
0,Contradiction,202,321,523,273006,0.084308
1,Entailment,325,356,681,463080,0.143005
2,Neutral,373,223,596,354620,0.109511


Compute the chance agreement as the sum of the last column and then insert into the formula. It is the same chance agreement as in the case of the coincidence matrix approach.

In [ ]:
Chance_agreement2=0.084308+0.143005+0.109511
Chance_agreement2

0.336824

### Fleiss Kappa (3+ raters)

This is the same as doing multiple Scott's Pi for pairs of raters and averaging the results

**Importing data**

In [54]:
#from google.colab import drive
#drive.mount('/content/drive')
Evaluator1 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Statistics/Curs/Data_c2/SNLI_Anca.xlsx')
Evaluator2 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Statistics/Curs/Data_c2/SNLI_Ionut.xlsx')
Evaluator3 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Statistics/Curs/Data_c2/SNLI_Larisa.xlsx')

In [55]:
Aggregated_kappa = pd.concat([Evaluator1['Label'], Evaluator2['Label'], Evaluator3['Label']], axis=1)

Aggregated_kappa = Aggregated_kappa.set_axis(['Evaluator1', 'Evaluator2', 'Evaluator3'], axis=1)

Aggregated_kappa

,Evaluator1,Evaluator2,Evaluator3
0,Neutral,Contradiction,Contradiction
1,Neutral,Neutral,Neutral
2,Entailment,Entailment,Entailment
3,Contradiction,Contradiction,Neutral
4,Neutral,Neutral,Entailment
...,...,...,...
895,Entailment,Entailment,Neutral
896,Neutral,Neutral,Entailment
897,Neutral,Entailment,Entailment
898,Contradiction,Contradiction,Contradiction


**Turning data into an appropiate format**

In [56]:
#!pip install statsmodels
from statsmodels.stats.inter_rater import aggregate_raters
Aggregated_kappa_subject_label = aggregate_raters(Aggregated_kappa, n_cat=None)
Aggregated_kappa_subject_label
df = pd.DataFrame(Aggregated_kappa_subject_label[0], columns=Aggregated_kappa_subject_label[1])
df['Sentence Pair'] = [f'Sentence Pair {i+1}' for i in range(len(df))]

# Reorder columns to have 'Sentence Pair' as the first column
df = df[['Sentence Pair', 'Contradiction', 'Entailment', 'Neutral']]

df

,Sentence Pair,Contradiction,Entailment,Neutral
0,Sentence Pair 1,2,0,1
1,Sentence Pair 2,0,0,3
2,Sentence Pair 3,0,3,0
3,Sentence Pair 4,2,0,1
4,Sentence Pair 5,0,1,2
...,...,...,...,...
895,Sentence Pair 896,0,2,1
896,Sentence Pair 897,0,1,2
897,Sentence Pair 898,0,2,1
898,Sentence Pair 899,3,0,0


❗ Notice that the "aggregate_rateres" produces a touple of two arrays so we need to select the first one

In [57]:
Aggregated_kappa_subject_label_good = Aggregated_kappa_subject_label[0]

**Compute Fleiss Kappa**

In [ ]:
#!pip install statsmodels
from statsmodels.stats.inter_rater import fleiss_kappa
fleiss_kappa(Aggregated_kappa_subject_label_good, method='fleiss')

0.49906026144937043

#### Fleiss Kappa with missing data ❓

Assuming a table with evaluated objects on rows and raters on columns.
Missing values coded as Nan

In [58]:
#from google.colab import drive
#drive.mount('/content/drive')
Evaluator1_Missing = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Statistics/Curs/Data_c2/Missing_SNLI_Anca.xlsx')
Evaluator2_Missing = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Statistics/Curs/Data_c2/Missing_SNLI_Ionut.xlsx')
Evaluator3_Missing = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Statistics/Curs/Data_c2/Missing_SNLI_Larisa.xlsx')
Aggregated_Missing_3 = pd.concat([Evaluator1_Missing['Label'],Evaluator2_Missing['Label'],Evaluator3_Missing['Label']], axis=1)

Aggregated_Missing_3 = Aggregated_Missing_3.set_axis([ 'Evaluator1', 'Evaluator2', 'Evaluator3'], axis=1)
Aggregated_Missing_3

,Evaluator1,Evaluator2,Evaluator3
0,Neutral,Contradiction,Contradiction
1,Neutral,Neutral,Neutral
2,Entailment,Entailment,Entailment
3,Contradiction,Contradiction,Neutral
4,Neutral,Neutral,Entailment
...,...,...,...
895,Entailment,Entailment,Neutral
896,Neutral,Neutral,Entailment
897,Neutral,Entailment,Entailment
898,Contradiction,Contradiction,Contradiction


**Check missing and replace with a string**

In [59]:
Aggregated_Missing_3.isna().sum()

,0
Evaluator1,15
Evaluator2,15
Evaluator3,8


In [60]:
Aggregated_Missing_3.fillna('Missing', inplace=True)

**Get rid of the missing column and turn data into an appropiate format**

In [61]:
#!pip install statsmodels
from statsmodels.stats.inter_rater import aggregate_raters
Formated = aggregate_raters(Aggregated_Missing_3, n_cat=None)
Formated

(array([[2, 0, 0, 1],
        [0, 0, 0, 3],
        [0, 3, 0, 0],
        ...,
        [0, 2, 0, 1],
        [3, 0, 0, 0],
        [3, 0, 0, 0]]),
 array(['Contradiction', 'Entailment', 'Missing', 'Neutral'], dtype=object))

In [62]:
missing_index = np.where(Formated[1] == 'Missing')[0][0]
Aggregated_Missing_3_formated = np.delete(Formated [0], missing_index, axis=1)
Aggregated_Missing_3_formated

array([[2, 0, 1],
       [0, 0, 3],
       [0, 3, 0],
       ...,
       [0, 2, 1],
       [3, 0, 0],
       [3, 0, 0]])

**Define a new function that only considers subjects that have at least 2 raters**

This is required as getting rid the "Missing" column above may leave subjects which are rated by only 1 or 0 raters

In [63]:
def fleiss_kappa_missing(table, method='fleiss'):

    table = 1.0 * np.asarray(table)   #avoid integer division
    n_sub, n_cat =  table.shape
    n_total = table.sum()

    #marginal frequency  of categories
    p_cat = table.sum(0) / n_total


    row_sums = table.sum(1)
    valid_rows = row_sums >= 2
    filtered_table = table[valid_rows]
    table2 = filtered_table * filtered_table
    p_rat = (table2.sum(1) - filtered_table.sum(1)) / (filtered_table.sum(1)*(filtered_table.sum(1)-1))
    p_mean = p_rat.mean()

    if method == 'fleiss':
        p_mean_exp = (p_cat*p_cat).sum()
    elif method.startswith('rand') or method.startswith('unif'):
        p_mean_exp = 1 / n_cat

    kappa = (p_mean - p_mean_exp) / (1- p_mean_exp)
    return kappa

**Compute Fleiss Kappa with missing data**

In [64]:
fleiss_kappa_missing(Aggregated_Missing_3_formated, method='fleiss')

0.5012577287595372

**Easier alternative** 🪶

Notice however that here we treat missing as a category


In [65]:
!pip install irrCAC

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.3/213.3 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1


In [66]:
from irrCAC.raw import CAC
cac_3raters_missing = CAC(Aggregated_Missing_3)
cac_3raters_missing.fleiss()

{'est': {'coefficient_value': 0.48703,
  'coefficient_name': "Fleiss' kappa",
  'confidence_interval': (0.45269, 0.52136),
  'p_value': 0.0,
  'z': 27.83913,
  'se': 0.01749,
  'pa': 0.66296,
  'pe': 0.34297},
 'weights': array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]),
 'categories': ['Contradiction', 'Entailment', 'Missing', 'Neutral']}

### Cronger's Kappa (3+ raters)

This is the same as doing multiple Cohen's Kappa for pairs of raters and averaging the results

**Import data**

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
Evaluator1 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Statistics/Curs/Data_c2/SNLI_Anca.xlsx')
Evaluator2 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Statistics/Curs/Data_c2/SNLI_Ionut.xlsx')
Evaluator3 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Statistics/Curs/Data_c2/SNLI_Larisa.xlsx')
Aggregated_cronger = pd.concat([Evaluator1['Label'], Evaluator2['Label'], Evaluator3['Label']], axis=1)

Aggregated_cronger = Aggregated_cronger.set_axis(['Evaluator1', 'Evaluator2', 'Evaluator3'], axis=1)

Aggregated_cronger

,Evaluator1,Evaluator2,Evaluator3
0,Neutral,Contradiction,Contradiction
1,Neutral,Neutral,Neutral
2,Entailment,Entailment,Entailment
3,Contradiction,Contradiction,Neutral
4,Neutral,Neutral,Entailment
...,...,...,...
895,Entailment,Entailment,Neutral
896,Neutral,Neutral,Entailment
897,Neutral,Entailment,Entailment
898,Contradiction,Contradiction,Contradiction


In [ ]:
#!pip install irrCAC
from irrCAC.raw import CAC
cac_3raters = CAC(Aggregated_cronger)
cac_3raters.conger()

{'est': {'coefficient_value': 0.50742,
  'coefficient_name': "Conger's kappa",
  'confidence_interval': (0.47426, 0.54058),
  'p_value': 0.0,
  'z': 30.03057,
  'se': 0.0169,
  'pa': 0.67556,
  'pe': 0.34134},
 'weights': array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]),
 'categories': ['Contradiction', 'Entailment', 'Neutral']}

#### Cronger's Kappa by hand 🤚

Here we show that chance agreement can be computed via a shortcurt using the average relative frequency and variance of each category.

**Percent agreement**


In [ ]:
from statsmodels.stats.inter_rater import aggregate_raters
Aggregated_subject_label = aggregate_raters(Aggregated_cronger, n_cat=None)
Aggregated_subject_label
df_subject_label = pd.DataFrame(Aggregated_subject_label[0], columns=Aggregated_subject_label[1])
df_subject_label['Sentence Pair'] = [f'Sentence Pair {i+1}' for i in range(len(df_subject_label))]

# Reorder columns to have 'Sentence Pair' as the first column
df_subject_label = df_subject_label[['Sentence Pair', 'Contradiction', 'Entailment', 'Neutral']]

df_subject_label

,Sentence Pair,Contradiction,Entailment,Neutral
0,Sentence Pair 1,2,0,1
1,Sentence Pair 2,0,0,3
2,Sentence Pair 3,0,3,0
3,Sentence Pair 4,2,0,1
4,Sentence Pair 5,0,1,2
...,...,...,...,...
895,Sentence Pair 896,0,2,1
896,Sentence Pair 897,0,1,2
897,Sentence Pair 898,0,2,1
898,Sentence Pair 899,3,0,0


This is a function to compute precent agreement in cases of 3+ raters with missing data

In [ ]:
data = df_subject_label.copy()

# Create a DataFrame
df = pd.DataFrame(data)

# Calculate the number of sentence pairs with at least two ratings
n_prime = (df[['Contradiction', 'Entailment', 'Neutral']].sum(axis=1) >= 2).sum()

# Initialize Po sum
Po_sum = 0

# Iterate over each sentence pair
for i, row in df.iterrows():
    # Initialize r_i sum for the current row
    r_i_sum = row['Contradiction'] + row['Entailment'] + row['Neutral']

    # Skip if r_i is less than 2
    if r_i_sum < 2:
        continue

    # Initialize sum for the current sentence pair
    pair_sum = 0

    # Iterate over each category (Contradiction, Entailment, Neutral)
    for k in ['Contradiction', 'Entailment', 'Neutral']:
        r_ik = row[k]

        # Calculate the inner sum for the current category
        if r_i_sum > 1:
            inner_sum = (r_ik * (r_ik - 1)) / (r_i_sum * (r_i_sum - 1))
        else:
            inner_sum = 0

        pair_sum += inner_sum

    # Add the pair_sum to Po_sum
    Po_sum += pair_sum

# Calculate Po
Po = Po_sum / n_prime

print(f'Po: {Po}')


Po: 0.6755555555555548


**Chance Agreement**

In [ ]:
counts = Aggregated_cronger.apply(pd.Series.value_counts).fillna(0).astype(int)

categories = ['Contradiction', 'Neutral', 'Entailment']
counts = counts.reindex(categories).fillna(0).astype(int)
counts = counts.transpose()
counts

,Contradiction,Neutral,Entailment
Evaluator1,202,373,325
Evaluator2,321,223,356
Evaluator3,206,172,522


Get the average probability for each category

In [ ]:
relative_counts = counts/900
averages = relative_counts.mean()
averages

Contradiction    0.270000
Neutral          0.284444
Entailment       0.445556
dtype: float64

Get the variace for each category

In [ ]:
variances_table = relative_counts.sub(averages).pow(2)
variances = variances_table.sum()/2
variances

Contradiction    0.005638
Neutral          0.013478
Entailment       0.013853
dtype: float64

Compute chance agreement

In [ ]:
Pe = averages.pow(2).sum() - (variances/3).sum() #3 is the number of raters

**Plug in into formula**

Small difference due to approximations

In [ ]:
(Po-Pe)/(1-Pe)

0.5046635180031721

### Intra-class correlation

**Preparing the data**

In [67]:
electro_dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Statistics/Curs/Data_c2/electromyography.csv')

In [68]:
electro_dataset

,Subject,Day 1,Day 2,Day 3
0,1,59.9,67.7,72.2
1,2,62.9,66.5,67.9
2,3,58.9,50.1,47.9
3,4,46.8,50.0,53.9
4,5,62.5,67.8,62.6
5,6,44.8,42.7,48.4
6,7,57.3,49.6,48.0
7,8,49.0,45.2,57.5
8,9,43.5,41.5,47.4
9,10,39.2,50.9,56.3


In [69]:
long_format_electro_dataset = pd.melt(electro_dataset, id_vars=['Subject'], var_name='Day', value_name='Value')
long_format_electro_dataset.head(15)

,Subject,Day,Value
0,1,Day 1,59.9
1,2,Day 1,62.9
2,3,Day 1,58.9
3,4,Day 1,46.8
4,5,Day 1,62.5
5,6,Day 1,44.8
6,7,Day 1,57.3
7,8,Day 1,49.0
8,9,Day 1,43.5
9,10,Day 1,39.2


**Applying the ICC**

In [70]:
!pip install pingouin
import pingouin as pg
icc = pg.intraclass_corr(data=long_format_electro_dataset, targets='Subject', raters='Day',
                         ratings='Value').round(3)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.4/204.4 kB 5.2 MB/s eta 0:00:00


In [71]:
icc

,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,0.706,8.202,9,20,0.0,"[0.39, 0.91]"
1,ICC2,Single random raters,0.708,8.696,9,18,0.0,"[0.39, 0.91]"
2,ICC3,Single fixed raters,0.720,8.696,9,18,0.0,"[0.4, 0.91]"
3,ICC1k,Average raters absolute,0.878,8.202,9,20,0.0,"[0.65, 0.97]"
4,ICC2k,Average random raters,0.879,8.696,9,18,0.0,"[0.66, 0.97]"
5,ICC3k,Average fixed raters,0.885,8.696,9,18,0.0,"[0.66, 0.97]"


####ICC by hand 🤚

In [72]:
electro_dataset_hand = electro_dataset.copy()
electro_dataset_hand

,Subject,Day 1,Day 2,Day 3
0,1,59.9,67.7,72.2
1,2,62.9,66.5,67.9
2,3,58.9,50.1,47.9
3,4,46.8,50.0,53.9
4,5,62.5,67.8,62.6
5,6,44.8,42.7,48.4
6,7,57.3,49.6,48.0
7,8,49.0,45.2,57.5
8,9,43.5,41.5,47.4
9,10,39.2,50.9,56.3


##### First partition the variances and get MSBS, MSWS, MSE, AND MSBM

**Compute Grand mean: the mean of all scores**

In [73]:
GrandMean = electro_dataset_hand[['Day 1', 'Day 2', 'Day 3']].values.mean()
GrandMean

53.96333333333333

**Compute Subject mean: mean of scores for each evaluated subject/object**

In [74]:
RaterMeans = electro_dataset_hand[['Day 1', 'Day 2', 'Day 3']].mean(axis=1)
RaterMeans
#RaterMeans = RaterMeans.to_numpy()
#RaterMeans

,0
0,66.600000
1,65.766667
2,52.300000
3,50.233333
4,64.300000
5,45.300000
6,51.633333
7,50.566667
8,44.133333
9,48.800000


**Compute Measurement mean: mean of scores for each evaluated subject/object**

In [75]:
MeasurementMeans = electro_dataset_hand[['Day 1', 'Day 2', 'Day 3']].mean(axis=0)
MeasurementMeans
#MeasurementMeans  = MeasurementMeans.to_numpy()
#MeasurementMeans

,0
Day 1,52.48
Day 2,53.20
Day 3,56.21


**Compute Mean Total Variation (MST)**

In [76]:
squared_diff_mean = (electro_dataset_hand[['Day 1', 'Day 2', 'Day 3']].sub(GrandMean)) ** 2
squared_diff_mean

,Day 1,Day 2,Day 3
0,35.244011,188.696011,332.576011
1,79.864011,157.168011,194.230678
2,24.370678,14.925344,36.764011
3,51.313344,15.708011,0.004011
4,72.874678,191.453344,74.592011
5,83.966678,126.862678,30.950678
6,11.133344,19.038678,35.561344
7,24.634678,76.796011,12.508011
8,109.481344,155.334678,43.077344
9,217.956011,9.384011,5.460011


In [77]:
SST=squared_diff_mean.sum().sum()
SST

2431.929666666667

In [78]:
MST=SST/(len(squared_diff_mean)*squared_diff_mean.shape[1]-1) # first is the number of subjects, second is the number of measurements/raters, -1 is to account for the degrees of freedom used to compute the Grand mean
MST

83.85964367816092

**Compute Mean variation of each rating from its Subject mean (MSWS)**

In [79]:
squared_diff_row_mean = (electro_dataset_hand[['Day 1', 'Day 2', 'Day 3']].sub(RaterMeans, axis=0)) ** 2
squared_diff_row_mean

,Day 1,Day 2,Day 3
0,44.890000,1.210000,31.360000
1,8.217778,0.537778,4.551111
2,43.560000,4.840000,19.360000
3,11.787778,0.054444,13.444444
4,3.240000,12.250000,2.890000
5,0.250000,6.760000,9.610000
6,32.111111,4.134444,13.201111
7,2.454444,28.801111,48.071111
8,0.401111,6.934444,10.671111
9,92.160000,4.410000,56.250000


In [80]:
SSWS = squared_diff_row_mean.sum().sum()
SSWS

518.4133333333332

In [81]:
MSWS=SSWS/(len(squared_diff_row_mean)*(squared_diff_row_mean.shape[1]-1)) # first is the number of subjects, second is the number of measurements/raters, -1 is to account for the degrees of freedom used to compute the subject means
MSWS

25.92066666666666

**Compute Mean variation of each rating from its Measurement/Rater mean (MSWM)**

In [82]:
squared_diff_col_mean = (electro_dataset_hand[['Day 1', 'Day 2', 'Day 3']].sub(MeasurementMeans, axis=1)) ** 2
squared_diff_col_mean

,Day 1,Day 2,Day 3
0,55.0564,210.25,255.6801
1,108.5764,176.89,136.6561
2,41.2164,9.61,69.0561
3,32.2624,10.24,5.3361
4,100.4004,213.16,40.8321
5,58.9824,110.25,60.9961
6,23.2324,12.96,67.4041
7,12.1104,64.00,1.6641
8,80.6404,136.89,77.6161
9,176.3584,5.29,0.0081


In [83]:
SSWM = squared_diff_col_mean.sum().sum()
SSWM

2353.6250000000005

In [84]:
MSWM=SSWM/(squared_diff_row_mean.shape[1]*(len(squared_diff_row_mean)-1)) # first is the number of measurements/raters,second is the number of subjects, last is -1 is to account for the degrees of freedom used to compute the measurement means
MSWM

87.17129629629632

**Compute mean variation of each Subject mean from Grand Mean (MSBS)**

In [85]:
squared_diff_subject_mean = (RaterMeans - GrandMean) ** 2 * squared_diff_row_mean.shape[1]
squared_diff_subject_mean

,0
0,479.056033
1,417.956033
2,8.300033
3,41.738700
4,320.540033
5,225.160033
6,16.286700
7,34.612033
8,289.886700
9,79.980033


In [86]:
SSBS = squared_diff_subject_mean.sum()
SSBS

1913.5163333333342

In [87]:
MSBS=SSBS/(len(squared_diff_row_mean)-1) # first term representing the number of subjects and -1 the lost degree of freedom from using the grand mean to compute variance
MSBS

212.61292592592602

**Compute mean variation of each Measurement mean from Grand Mean (MSBM)**

In [88]:
squared_diff_rater_mean = (MeasurementMeans - GrandMean) ** 2 * len(squared_diff_row_mean)
squared_diff_rater_mean

,0
Day 1,22.002778
Day 2,5.826778
Day 3,50.475111


In [89]:
SSBM = squared_diff_rater_mean.sum()
SSBM

78.30466666666655

In [90]:
MSBM=SSBM/(squared_diff_row_mean.shape[1]-1) # first term representing the number of raters/measurements and -1 the lost degree of freedom from using the grand mean to compute variance
MSBM

39.152333333333274

**Compute Mean Error for each measurement (MSE)**

In [91]:
SSE=SST-SSBS-SSBM
SSE

440.1086666666662

In [92]:
MSE=SSE/((len(squared_diff_row_mean)-1)*(squared_diff_row_mean.shape[1]-1)) # first is the number of subjects, second is the number of measurements/raters, -1 is to account for the degrees of freedom used to compute the subject means
MSE

24.450481481481454

##### Then plug these into the formulas
Check: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6645485/#pone.0219854.s002 to see derivations of formulas


**Plug values into the formula of ICC1**

In [93]:
(MSBS-MSWS)/(MSBS+(squared_diff_row_mean.shape[1]-1)*MSWS)

0.7059529303184128

**Plug values into the formula of ICC2**

In [94]:
(MSBS-MSE)/(MSBS+(squared_diff_row_mean.shape[1]-1)*MSE+(squared_diff_row_mean.shape[1]/len(squared_diff_row_mean))*(MSBM-MSE))

0.7075785937526118

**Plug values into the formula of ICC3**

In [95]:
(MSBS-MSE)/(MSBS+(squared_diff_row_mean.shape[1]-1)*MSE)

0.7195122417547404

**Plug values into the formula of ICC1k**

In [96]:
(MSBS-MSWS)/(MSBS)

0.8780851796579039

**Plug values into the formula of ICC2k**

In [97]:
(MSBS-MSE)/ (MSBS+(MSBM-MSE)/len(squared_diff_row_mean))

0.8789224122718696

**Plug values into the formula of ICC3k**

In [98]:
(MSBS-MSE)/(MSBS)

0.8850000235169146